In [1]:
import sys, os
import pandas as pd
import numpy as np
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt

sys.path.insert(1, '../scripts')
root = os.getcwd()

from TwintDataMiner import TwintDataMiner
from PandasDataCleaner import PandasDataCleaner
from TextBlobSentiment import TextBlobSentiment
from Visualisation import Visualisation
from GMM import GMM


In [2]:
tweetsFile = "../data/1000TweetsPerDay.csv"
tdm = TwintDataMiner()
attributes = ["id", "tweet", "place", "date"]
df = tdm.getPandasDataFrame(attributes, tweetsFile)
df.shape
df.head

<bound method NDFrame.head of                         id                                              tweet  \
0                       id                                              tweet   
1      1212480089155354628  El M*gnici*io es la única opción para Venezuel...   
2      1212479875220566016  plying to @ironorehopper\nInteresting, further...   
3      1212475589325676544  Bienvenidos a los Fabulosos años 20 con @Mierc...   
4      1212473830758899715  Meu Deus, tão inventando nome pra tudo "White ...   
...                    ...                                                ...   
89952  1247297335035641859  Wisconsin Supreme Court, on primary eve, block...   
89953  1247297334741999616  What is becoming of us?    Teen Girls Attack #...   
89954  1247297333743751168  Why does every article about this talks about ...   
89955  1247297333676605440  Recado para senhora corona vírus:\n\nse vc se ...   
89956  1247297333601153024  Schumer names coronavirus czar candidates in p...  

In [3]:
df.drop_duplicates(subset='id')
df.shape

(89957, 4)

In [4]:
def cleandf(df):
    pdc = PandasDataCleaner()
    
    df  = pdc.detectLanguage(df, 'tweet', 'lang') 
    df = df.loc[df['lang'] == 'en']
    df = pdc.cleanTweets(df, 'tweet')


#     df = df.reset_index(drop=True)

    return df

In [14]:
from LDA import LDA
from pprint import pprint

lda = LDA()

lemmatised_vocab = pd.DataFrame()
lemmatised_vocab['column'] = [['death', 'ventil', 'crisi', 'scar', 'tragedi', 'struggl']]

lda.dict_from_vocab(doc=lemmatised_vocab.iloc[0])


def quickting():
    bow_corpus = []
    counter = 0
    for j in range(1,5):
        for i in range(1,32):
            if(not((j == 1 and i < 10) or (j == 2 and i > 29) or (j == 4 and i > 6))):
                print(counter)
                if counter >= 20:
                    return bow_corpus
                date = "2020-0" + str(j) + "-" + str(i).zfill(2)
                tempdf = df[df.date == date].copy()
                tempdf = tempdf.reset_index(drop=True)

                tempdf.sort_values("id", inplace=True) 

                # cleaning and dropping duplicate values
                tempdf.drop_duplicates(subset=["tweet"],inplace=True)
                tempdf = cleandf(tempdf)
                tempdf = tempdf[:250]

                # lemmatise and tokenise
                processed_docs = tempdf['tweet'].map(str).map(lda.preprocess)

                tempdf['Lemmatised'] = processed_docs
                bow_corpus.extend(lda.generateBagOfWords(docs=processed_docs))
                counter +=1
    return bow_corpus

bow_corpus = quickting()
corpus_tfidf = lda.tf_idf(bow_corpus=bow_corpus)
print("------------ BOW ------------")
lda_model_bow = lda.train(bow_corpus, no_topics, dictionary)
for idx, topic in lda_model_bow.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
print("\n\n------------ TFIDF ------------")
lda_model_tfidf = lda.train(corpus_tfidf, no_topics, dictionary, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
------------
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(0, 1.0)]
[]
[(0, 1.0)]
[]
[]
[]
[]
[(0, 1.0)]
[(0, 1.0)]
[]
[]
[]
[(0, 1.0)]
[(0, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(0, 1.0)]
[(0, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(0, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[(0, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(3, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(0, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(1, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(1, 1.0)]
[]
[]
[]
[]
[]
[]
[(1, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(1, 1.0)]
[]
[(1, 1.0)]
[]
[]
[]
[]
[]


[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(1, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(0, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(1, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[(2, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(1, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(1, 1.0)]
[]
[(1, 1.0)]
[]
[]
[]
[(1, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(1, 1.0)]
[]
[(1, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(0, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[(1, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(1, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(1, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(1, 1.0)]
[]
[]
[(1, 1.0)]
[]
[]
[(1, 1.0)]
[]
[]
[]
[(1, 1.0)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(1, 1.0)]
[]
[]
[]
[]
[]
[]
[]


In [11]:
vocab = ['death', 'deaths', 'ventilator', 'ventilators', 'crisis', 'scared', 'tragedy', 'struggling']
bowMatrix = np.empty((1, len(vocab)), int)
gmm = GMM()
#generate bag of words for each day and add to matrix
for j in range(1,5):
    for i in range(1,32):
        if(not((j == 1 and i < 10) or (j == 2 and i > 29) or (j == 4 and i > 6))):
            date = "2020-0" + str(j) + "-" + str(i).zfill(2)
#             print(date)
            tempdf = df[df.date == date].copy()
            tempdf = tempdf.reset_index(drop=True)

            tempdf.sort_values("id", inplace=True) 
  
            # dropping duplicate values 
            tempdf.drop_duplicates(subset=["tweet"],inplace=True)
            pdc = PandasDataCleaner()
            tempdf  = pdc.detectLanguage(tempdf, 'tweet', 'lang')
            tempdf = tempdf.loc[tempdf['lang'] == 'en']
            tempdf = pdc.cleanTweets(tempdf, 'tweet')
            tempdf = tempdf[:250]
            
            bow = gmm.generateBagOfWords(tempdf, 'tweet', vocab=vocab)
            #get total bag of words for day
            panicRow = np.sum(bow, axis=0)
            bowMatrix = np.append(bowMatrix, np.reshape(panicRow, (1, len(vocab))), axis=0)
#             print(bowMatrix)
            
#delete row created for initialisation          
bowMatrix = np.delete(bowMatrix, (0), axis=0)
print(bowMatrix.shape)
print(bowMatrix)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [

KeyboardInterrupt: 

In [7]:
panic = gmm.modelFromBow(bowMatrix)

NameError: name 'gmm' is not defined

In [8]:
print(panic)

NameError: name 'panic' is not defined

In [9]:
onePanic = panic[:, 0]
print(onePanic)

NameError: name 'panic' is not defined

In [10]:
plt.plot(onePanic)
plt.show()

NameError: name 'plt' is not defined